In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms


from torchvision.models import vit_b_16
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time

import argparse
import os
import copy
import dataloader


/usr/scratch/abhimanyu/work/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--annot_train_prime', type = str, default = 'df_prime_train.csv')
    parser.add_argument('--annot_test_prime', type = str, default = 'df_prime_test.csv')
    parser.add_argument('--data_root', type = str, default = '/usr/scratch/abhimanyu/courses/ECE8803_FML/OLIVES')
    parser.add_argument('--lr', type = int, default = 0.001)
    parser.add_argument('--momentum', type = int, default = 0.9)
    parser.add_argument('--epoch', type = int, default = 50)
    parser.add_argument('--batch_size', type = int, default = 5)
    parser.add_argument('--save_pth', type = str, default = '/usr/scratch/yangyu/FML_Model/vit')
  

    return parser.parse_known_args()

In [4]:

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args, unkown = parse_args()
# Define transform
timestr = time.strftime("%Y%m%d-%H%M%S")
base_name = "restnet18_" + timestr + ".pth" 
name = os.path.join(args.save_pth, base_name)
args.save_pth = os.path.abspath(name)

# # Load dataset
# train_dataset = datasets.ImageFolder(root='path_to_train_folder', transform=transform)
# test_dataset = datasets.ImageFolder(root='path_to_test_folder', transform=transform)

# # Define dataloader
batched_trainset, batched_testset = dataloader.dataloader(args, 'ResNet')
num_classes 


In [8]:
print(len(batched_trainset), len(batched_testset))

4851 1598


In [9]:
# Define model
model = vit_b_16()

# Define optimizer and loss function

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    
k = model.fc.in_features
model.fc = nn.Linear(k, num_class)
model = model.to(device)

# Tensorboard Writer
writer = SummaryWriter('path_to_tensorboard_logs')

# Train model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0.0
    total_predictions = 0.0
    start_time = time.time()

    for images, labels in tqdm(batched_trainset, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record training loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    # Evaluate model on test data
    test_loss = 0.0
    test_correct_predictions = 0.0
    test_total_predictions = 0.0
    with torch.no_grad():
        for images, labels in batched_testset:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Record test loss and accuracy
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total_predictions += labels.size(0)
            test_correct_predictions += (predicted == labels).sum().item()

    # Print statistics and add to Tensorboard
    end_time = time.time()
    epoch_time = end_time - start_time
    train_loss = running_loss / len(batched_trainset)
    train_accuracy = correct_predictions / total_predictions
    test_loss /= len(batched_testset)
    test_accuracy = test_correct_predictions

Epoch 1/10:   6%|▋         | 310/4851 [02:22<34:50,  2.17it/s] 

In [5]:
print(vit_b_16().to(device))

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

: 